In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - Inference Demo

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

This notebook shows how to use Feature Store and Model Endpoint for Realtime Inference

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)

Steps performed in this notebook:

* Invoke a Vetex AI Feature Store and Vetex AI Endpoint to test Realtime predictions

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

### Import libraries and define constants

#### Libraries
Next you will import the libraries needed for this notebook. 

Note that currently this notebook uses KFP SDK v1, whereas the environment includes KFP v2. As an interim solution, we will downlevel KFP and the Google Cloud Pipeline Components in order to use the v1 code here as-is. See the [KFP migration guide](https://www.kubeflow.org/docs/components/pipelines/v2/migration/) for more details of moving from v1 to v2. 

#### Variables

#### Initialize the Vertex AI SDK
Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
REGION="us-central1"

In [ ]:
# General imports
import os
import sys
import random
from datetime import datetime, timedelta
import json
from google.cloud import aiplatform as vertex_ai

In [ ]:
# Vertex AI SDK
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [ ]:
# Define helper methods

In [ ]:
def read_from_sub(project_id, subscription_name, messages=10):
    """
    Read messages from a Pub/Sub subscription
    Args:
        project_id: project ID
        subscription_name: the name of a Pub/Sub subscription in your project
        messages: number of messages to read
    Returns:
        msg_data: list of messages in your Pub/Sub subscription as a Python dictionary
    """
    
    import ast

    from google.api_core import retry
    from google.cloud import pubsub_v1

    subscriber = pubsub_v1.SubscriberClient()
    subscription_path = subscriber.subscription_path(project_id, subscription_name)

    # Wrap the subscriber in a 'with' block to automatically call close() to
    # close the underlying gRPC channel when done.
    with subscriber:
        # The subscriber pulls a specific number of messages. The actual
        # number of messages pulled may be smaller than max_messages.
        response = subscriber.pull(
            subscription=subscription_path,
            max_messages=messages,
            retry=retry.Retry(deadline=300),
        )

        if len(response.received_messages) == 0:
            print("no messages")
            return

        ack_ids = []
        msg_data = []
        for received_message in response.received_messages:
            msg = ast.literal_eval(received_message.message.data.decode("utf-8"))
            msg_data.append(msg)
            ack_ids.append(received_message.ack_id)

        # Acknowledges the received messages so they will not be sent again.
        subscriber.acknowledge(subscription=subscription_path, ack_ids=ack_ids)

        print(
            f"Received and acknowledged {len(response.received_messages)} messages from {subscription_path}."
        )

        return msg_data

In [ ]:
from google.cloud.aiplatform import Featurestore, EntityType, Feature

def features_lookup(ff_feature_store, entity, entity_ids):
    '''
    Function that retrieves feature values from Vertex AI Feature Store
    '''
    entity_type = ff_feature_store.get_entity_type(entity)
    aggregated_features = entity_type.read(entity_ids=entity_ids,feature_ids="*")
    aggregated_features_preprocessed = preprocess(aggregated_features)
    features = aggregated_features_preprocessed.iloc[0].to_dict()
    return features

def preprocess(payload):
    '''
    Function that pre-processes the payload values
    '''
    # replace NaN's
    for key , value in payload.items():
        if value is None:
            payload[key] = 0.0
    return payload

In [ ]:
#ENDPOINT_ID="..."
#FEATURESTORE_ID='fraudfinder_...'

In [ ]:
print(ENDPOINT_ID)
from google.cloud import aiplatform as aiplatform
# Instantiate Vertex AI Endpoint object
endpoint_obj = aiplatform.Endpoint(ENDPOINT_ID)

In [ ]:
# Instantiate Vertex AI Feature Store object
try:
    ff_feature_store = Featurestore(FEATURESTORE_ID)
except NameError:
    print(f"""The feature store {FEATURESTORE_ID} does not exist!""")

In [ ]:
messages_tx = read_from_sub(
    project_id=PROJECT_ID, subscription_name="ff-tx-sub", messages=1
)

messages_tx

In [ ]:
import pprint
pp = pprint.PrettyPrinter(compact=True)

# Payload for manual test:
# payload_json = {
#     "TX_ID": "61210be0744c43232990152d3eb2c2deb6035d8b",
#     "TX_TS": "2025-09-06 17:27:51",
#     "CUSTOMER_ID": "7389471951168361",
#     "TERMINAL_ID": "45087784",
#     "TX_AMOUNT": 32.77
# }

# Reading 1-st message
payload_json = messages_tx[0]

payload={}
payload["tx_amount"] = payload_json["TX_AMOUNT"]

# look up the customer features from Vertex AI Feature Store
customer_features = features_lookup(ff_feature_store, "customer",[payload_json["CUSTOMER_ID"]])
# print the customer features from Vertex AI Feature Store
print("-------------------------------------------------------")
print("customer_features:")
pp.pprint(customer_features)

# look up the terminal features from Vertex AI Feature Store
terminal_features = features_lookup(ff_feature_store, "terminal",[payload_json["TERMINAL_ID"]])
print("-------------------------------------------------------")
print("terminal features:")
pp.pprint(terminal_features)

# Add customer features to payload
payload.update(customer_features)

# Add terminal features to payload
payload.update(terminal_features)

del payload['entity_id']

payload = preprocess(payload)

print("-------------------------------------------------------")
print("[Payload to be sent to Vertex AI endpoint]")
pp.pprint(payload)
print("-------------------------------------------------------")

result = endpoint_obj.predict(instances = [payload])

print("-------------------------------------------------------")
pp.pprint(f"[Prediction result]: {result}")
print("-------------------------------------------------------")